In [17]:
#Importamos las librerías que necesitamos, numpy y pandas para la limpieza y tratamiento de datos

import numpy as np
import pandas as pd

In [2]:
#Creamos las url como variables 

url_trip_data = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.csv"
url_taxi_zone = "https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv"

In [3]:
# Leemos los ficheros csv a través de las url
trip_taxi = pd.read_csv(url_trip_data)
taxi_zone = pd.read_csv(url_taxi_zone)

/opt/miniconda3/envs/dexs/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Convertimos en DataFrame
df_trip_taxi = pd.DataFrame(trip_taxi)
df_taxi_zone = pd.DataFrame(taxi_zone)

In [5]:
# Revisamos las columnas y tipos de datos de cada dataFrame
df_trip_taxi.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1369765 entries, 0 to 1369764
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1271413 non-null  float64
 1   tpep_pickup_datetime   1369765 non-null  object 
 2   tpep_dropoff_datetime  1369765 non-null  object 
 3   passenger_count        1271413 non-null  float64
 4   trip_distance          1369765 non-null  float64
 5   RatecodeID             1271413 non-null  float64
 6   store_and_fwd_flag     1271413 non-null  object 
 7   PULocationID           1369765 non-null  int64  
 8   DOLocationID           1369765 non-null  int64  
 9   payment_type           1271413 non-null  float64
 10  fare_amount            1369765 non-null  float64
 11  extra                  1369765 non-null  float64
 12  mta_tax                1369765 non-null  float64
 13  tip_amount             1369765 non-null  float64
 14  tolls_amount      

In [6]:
df_taxi_zone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [7]:
# Limpiamos el dataframe df_trip_taxi, quedándonos con las columnas que vamos a utilizar
df_trip_taxi= df_trip_taxi[["trip_distance", "DOLocationID"]]
df_trip_taxi.head()

,trip_distance,DOLocationID
0,2.10,43
1,0.20,151
2,14.70,165
3,10.60,132
4,4.94,33


In [8]:
# Comprobamos si hay nulos
df_trip_taxi.isnull().values.any()

False

In [9]:
# Nos quedamos con los valores que se encuentran por encima del percentil 95 con respecto a la distancia de viaje
percentile = 95
df_trip_taxi = df_trip_taxi.loc[df_trip_taxi['trip_distance'] > np.percentile (df_trip_taxi.trip_distance, percentile)]
df_trip_taxi

,trip_distance,DOLocationID
2,14.70,165
3,10.60,132
24,19.10,3
29,14.30,226
34,10.74,231
...,...,...
1369735,11.56,237
1369737,13.65,137
1369739,11.62,157
1369742,14.07,21


In [10]:
# Agrupamos por número de viajes por destino y los ordenamos de forma descendente
df_trip_taxi = df_trip_taxi.groupby(['DOLocationID']).size().reset_index(name = 'trips')
df_trip_taxi.sort_values(by='trips', ascending=False, inplace= True)
df_trip_taxi


,DOLocationID,trips
259,265,2101
127,132,1883
133,138,1111
60,61,996
88,89,904
...,...,...
7,8,7
1,2,3
29,30,2
247,253,2


In [11]:
# Nos quedamos con los 10 primeros datos
df_trip_taxi = df_trip_taxi.iloc[0:10]
df_trip_taxi

,DOLocationID,trips
259,265,2101
127,132,1883
133,138,1111
60,61,996
88,89,904
47,48,804
165,170,773
13,14,763
183,188,742
78,79,729


In [12]:
# Una vez hechas todas las modificaciones de la tabla que afectan al número de filas, reseteamos el índice
df_trip_taxi.reset_index(inplace=True, drop=True)

In [13]:
# Añadimos las columnas "end_boroug" y "end_zone" vacías, para luego rellenarlas con los datos de la tabla df_taxi_zone
df_trip_taxi.insert(2,'end_borough','')
df_trip_taxi.insert(3,'end_zone','')
df_trip_taxi.head()

,DOLocationID,trips,end_borough,end_zone
0,265,2101,,
1,132,1883,,
2,138,1111,,
3,61,996,,
4,89,904,,


In [14]:
# Creamos una función para buscar la localización a través del locationID en el dataFrame df_taxi_zone
def search_location_by_locationId(locationId,location):
    return df_taxi_zone.loc[df_taxi_zone['LocationID'] == locationId ,location].iloc[0]


In [15]:
# Iteramos la columna "DOLocationID" de "df_trip_taxi" para  \
# obtener sus localizaciones ("Zone" y "Borough") con el metodo "search_location_by_locationId" y persistirlas en "df_trip_taxi"

for index, row in df_trip_taxi.iterrows():
     locationID = row['DOLocationID']
     df_trip_taxi.end_borough[index] = search_location_by_locationId(locationID,'Borough')
     df_trip_taxi.end_zone[index] = search_location_by_locationId(locationID,'Zone')

df_trip_taxi

/var/folders/3p/6w0l61r57_s2_bg0xtykz8bh0000gn/T/ipykernel_39216/4230383224.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trip_taxi.end_borough[index] = search_location_by_locationId(locationID,'Borough')
/var/folders/3p/6w0l61r57_s2_bg0xtykz8bh0000gn/T/ipykernel_39216/4230383224.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trip_taxi.end_zone[index] = search_location_by_locationId(locationID,'Zone')


,DOLocationID,trips,end_borough,end_zone
0,265,2101,Unknown,NaN
1,132,1883,Queens,JFK Airport
2,138,1111,Queens,LaGuardia Airport
3,61,996,Brooklyn,Crown Heights North
4,89,904,Brooklyn,Flatbush/Ditmas Park
5,48,804,Manhattan,Clinton East
6,170,773,Manhattan,Murray Hill
7,14,763,Brooklyn,Bay Ridge
8,188,742,Brooklyn,Prospect-Lefferts Gardens
9,79,729,Manhattan,East Village


In [16]:
# Eliminamos la columna "DOLocation" para que la tabla quede conla estructura solicitada
df_trip_taxi.drop(["DOLocationID"], axis=1, inplace=True)
df_trip_taxi

/opt/miniconda3/envs/dexs/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,trips,end_borough,end_zone
0,2101,Unknown,NaN
1,1883,Queens,JFK Airport
2,1111,Queens,LaGuardia Airport
3,996,Brooklyn,Crown Heights North
4,904,Brooklyn,Flatbush/Ditmas Park
5,804,Manhattan,Clinton East
6,773,Manhattan,Murray Hill
7,763,Brooklyn,Bay Ridge
8,742,Brooklyn,Prospect-Lefferts Gardens
9,729,Manhattan,East Village
